# Specifying the requirement packages with versions necessary for the model.

In [ ]:
!pip install -U pip
!pip install numpy==1.21.6
!pip install scipy==1.7.3
!pip install pandas==1.5.2
!pip install scikit-learn==1.1.2
!pip install tensorflow==2.11.0
!pip install h5py==3.7.0
!pip install torch==1.12.1
!pip install -U pickle5

In [ ]:
!pip list | egrep 'matplotlib|numpy|pandas|scikit-learn|tensorflow|torch'

matplotlib                    3.2.2
matplotlib-venn               0.11.7
numpy                         1.21.6
pandas                        1.5.2
pandas-datareader             0.9.0
pandas-gbq                    0.17.9
pandas-profiling              1.4.1
scikit-learn                  1.1.2
sklearn-pandas                1.8.0
tensorflow                    2.11.0
tensorflow-datasets           4.6.0
tensorflow-estimator          2.11.0
tensorflow-gcs-config         2.9.1
tensorflow-hub                0.12.0
tensorflow-io-gcs-filesystem  0.28.0
tensorflow-metadata           1.11.0
tensorflow-probability        0.17.0
torch                         1.12.1+cu113
torchaudio                    0.12.1+cu113
torchsummary                  1.5.1
torchtext                     0.13.1
torchvision                   0.13.1+cu113


# Imports

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Conv2D, Activation, MaxPooling2D, Flatten, Dense)
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import pickle
import numpy as np
from keras.utils.np_utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset

## Loading data from pkl file

In [ ]:
pkl = open('/content/drive/MyDrive/Copy of train100c5k_v2.pkl', 'rb')
im = pickle.load(pkl)

## Preprocessing the data and creating training testing split

In [ ]:
data = im['data'].values
lable = im['target'].values

# print(lable.shape)

data = [x for x in data]
data = np.array(data)
data = data.reshape(500000,28,28,1)

label = LabelBinarizer()
lable = label.fit_transform(lable)

train_data, test_data, train_lable, test_lable = train_test_split(data, lable, test_size=0.05,random_state=420)

# scaleing the data
train_data = train_data / 255.0
test_data = test_data / 255.0


print("data shape",train_data.shape,test_data.shape)
print("lable shape",train_lable.shape,test_lable.shape)


data shape (475000, 28, 28, 1) (25000, 28, 28, 1)
lable shape (475000, 100) (25000, 100)


# Classifier Model

In [ ]:
class Im_classfification:

    def construct(width, height, depth, classes):
        model = Sequential()
        inputShape = (width, height, depth)

        if K.image_data_format() == "channels_first":
            inputShape = (depth, width, height)
            print(inputShape)

        # first set of convolution net
        print(inputShape,"asdasdsad")
        model.add(Conv2D(32, (5,5), strides=(1, 1),padding="valid", input_shape=inputShape))
        model.add(layers.BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # second set of convolution net
        model.add(Conv2D(50, (5, 5), strides=(1, 1), padding="same"))
        model.add(layers.BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # third layer of convolution net
        model.add(Conv2D(64, (5, 5), strides=(1, 1), padding="valid"))
        model.add(layers.BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # Fully connect layer
        model.add(Flatten())
        model.add(Dense(512))
        model.add(layers.BatchNormalization())
        model.add(Activation('relu'))
        model.add(layers.Dropout(0.25))

        # reduction the output
        model.add(Dense(256))
        model.add(layers.BatchNormalization())
        model.add(Activation('relu'))
        model.add(layers.Dropout(0.25))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation('softmax'))

        return model


## Compiling the model

In [ ]:
print("compiling model .......")
model = Im_classfification.construct(28,28,1,100)
# model = classifier.construct(28,28,1,100)
model.compile(optimizer='adam', loss="categorical_crossentropy",
            metrics=["accuracy"])

compiling model .......
(28, 28, 1) asdasdsad


# Training the model

In [ ]:
print("training the network .......")
H = model.fit(train_data, train_lable, batch_size=200, epochs=50, verbose=1, validation_data=(test_data,test_lable))

## Saving the Model for further utilization

In [ ]:
print("saving the model ........")
model.save("image_recogniation.h5")

saving the model ........


## Model Evaluation

In [ ]:
predictions = model.predict(test_data, batch_size=180)
print(classification_report(test_lable.argmax(axis=1), predictions.argmax(axis=1)))

139/139 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.63      0.83      0.72       249
           1       0.85      0.88      0.87       258
           2       0.70      0.68      0.69       252
           3       0.81      0.84      0.83       264
           4       0.80      0.75      0.77       248
           5       0.70      0.75      0.73       233
           6       0.76      0.70      0.73       264
           7       0.49      0.33      0.40       238
           8       0.76      0.79      0.78       245
           9       0.42      0.43      0.43       230
          10       0.58      0.59      0.58       252
          11       0.57      0.59      0.58       254
          12       0.71      0.78      0.74       246
          13       0.83      0.81      0.82       239
          14       0.87      0.89      0.88       230
          15       0.90      0.87      0.89       248
          16       0.80   

In [ ]:
!python /content/mini_project.py '/content/drive/MyDrive/Copy of train100c5k_v2.pkl' '/content/test100c5k_nolabel.pkl'

2022-12-05 04:31:00.332201: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 04:31:06.078179: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-05 04:31:06.079082: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-05 04:31:06.079123: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

## Github Quick draw: https://github.com/googlecreativelab/quickdraw-dataset